In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
train=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
train.head()
train_df_shuffled=train.sample(frac=1,random_state=42)
train_df_shuffled.head(3)

In [ ]:
test_df=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
test_df.head()

In [ ]:
train_df_shuffled.target.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [ ]:

import tensorflow as tf
import tensorflow_hub as hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string, 
                                        trainable=False,
                                        name="USE") 

In [ ]:
from tensorflow.keras import layers
model = tf.keras.Sequential([
  sentence_encoder_layer, 
  layers.Dense(256, activation="relu"),
  layers.Dense(64,activation='relu') , 
  layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model.summary()

In [ ]:
model_history = model.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

In [ ]:
preds=model.predict(test_df['text'])
preds[:5]

In [ ]:
pred=tf.squeeze(tf.round(preds))
pred=tf.round(pred.numpy())
pred.shape

In [ ]:
k=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
k

In [ ]:
my_submission = pd.DataFrame({'id': test_df['text'],'target':pred})
my_submission.to_csv('submission.csv', index=False)